# MUC_person_extraction

Extract Person entities from MUC (Message Understanding Conference) 3 corpus.

Compare to gold standard and calculate Coverage and Confidence measures.

References:
https://github.com/dstl/muc3


In [20]:
# Imports

import csv, re, itertools

import pandas as pd

import nltk

# Import spacy and English models
import spacy, numpy

# Load English Spacy module
nlp = spacy.load('en')

In [21]:
# Read in gold standard file

goldstandardfile_name = "C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\DSTL preprocessed data\\name-person_utf8.txt"
goldstandardfile_title = "C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\DSTL preprocessed data\\persons_utf8.txt"

goldstandard_name = open(goldstandardfile_name, 'r', encoding='utf-8', newline='\n').read()
goldstandard_title = open(goldstandardfile_title, 'r', encoding='utf-8', newline='\n').read()


# Clear spurious char strings
goldstandard_name = re.sub('\r', '', goldstandard_name)
goldstandard_title = re.sub('\r', '', goldstandard_title)


gold_name_list = goldstandard_name.split('\n')
gold_title_list = goldstandard_title.split('\n')



# Check length of files
print("Gold name file length= ", len(gold_name_list), "\nGold title file length = ", len(gold_title_list))

Gold name file length=  2233 
Gold title file length =  1537


In [22]:
# Convert to DataFrame and dedupe

goldstandard_name_df = pd.DataFrame(gold_name_list)
goldstandard_title_df = pd.DataFrame(gold_title_list)

goldstandard_name_df = goldstandard_name_df[0].drop_duplicates().sort_values().to_frame()
goldstandard_title_df = goldstandard_title_df[0].drop_duplicates().sort_values().to_frame()


# Prep columns
goldstandard_name_df.columns = ['Entity']
goldstandard_name_df['Entity'] = goldstandard_name_df['Entity'].str.strip()
goldstandard_name_df['Actual'] = 'Y'


#goldstandard_name_df
goldstandard_name_df.head(10)

,Entity,Actual
1,A. CRISTIANI,Y
2,ABDIEL ADAMES,Y
3,ABELARDO MATA,Y
4,ABELICA GARCIA SANCHEZ,Y
5,ABILIO DINIZ,Y
6,ABIMAEL GUZMAN,Y
7,ABRAHAM LINCOLN,Y
8,ACALARDO JARA SOTO,Y
9,ADALINO BARBERI,Y
10,ADAN SOLORZANO MARTINEZ,Y


In [23]:
# Read in 4 sample documents

samplefilepath1 = "C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\muc34\\TASK\\CORPORA\\tst1-muc3_utf8.txt"
samplefilepath2 = "C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\muc34\\TASK\\CORPORA\\tst2-muc4"
samplefilepath3 = "C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\muc34\\TASK\\CORPORA\\tst3-muc4"
samplefilepath4 = "C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\muc34\\TASK\\CORPORA\\tst4-muc4"

# with open(keyfilepath, 'r') as f:
#     reader = csv.reader(f)
#     keyfile = list(reader)

samplefile1 = open(samplefilepath1, 'r', encoding='utf8', newline='\n').read()
samplefile2 = open(samplefilepath2, 'r').read()
samplefile3 = open(samplefilepath3, 'r').read()
samplefile4 = open(samplefilepath4, 'r').read()

sample = samplefile1 + "\n" + samplefile2 + "\n" + samplefile3 + "\n" + samplefile4


# Clear spurious char strings
sample = re.sub('\r', '', sample)
sample = re.sub('\n', '', sample)


sample

'\ufeffTST1-MUC3-0001   GUATEMALA CITY, 4 FEB 90 (ACAN-EFE) -- [TEXT] THE GUATEMALA ARMYDENIED TODAY THAT GUERRILLAS ATTACKED THE "SANTO TOMAS" PRESIDENTIALFARM, LOCATED ON THE PACIFIC SIDE, WHERE PRESIDENT CEREZO HAS BEENSTAYING SINCE 2 FEBRUARY.   A REPORT PUBLISHED BY THE "CERIGUA" NEWS AGENCY -- MOUTHPIECE OFTHE GUATEMALAN NATIONAL REVOLUTIONARY UNITY (URNG) -- WHOSE MAINOFFICES ARE IN MEXICO, SAYS THAT A GUERRILLA COLUMN ATTACKED THE FARM2 DAYS AGO.   HOWEVER, ARMED FORCES SPOKESMAN COLONEL LUIS ARTURO ISAACS SAIDTHAT THE ATTACK, WHICH RESULTED IN THE DEATH OF A CIVILIAN WHO WASPASSING BY AT THE TIME OF THE SKIRMISH, WAS NOT AGAINST THE FARM, ANDTHAT PRESIDENT CEREZO IS SAFE AND SOUND.   HE ADDED THAT ON 3 FEBRUARY PRESIDENT CEREZO MET WITH THEDIPLOMATIC CORPS ACCREDITED IN GUATEMALA.   THE GOVERNMENT ALSO ISSUED A COMMUNIQUE DESCRIBING THE REBEL REPORTAS "FALSE AND INCORRECT," AND STRESSING THAT THE PRESIDENT WAS NEVERIN DANGER.   COL ISAACS SAID THAT THE GUERRILLAS ATTACKED THE 

In [56]:
# Read in sample as single text file

samplefilepath = "C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\muc34\\TASK\\CORPORA\\muc3_all.txt"

sample = open(samplefilepath, 'r', encoding='utf-8').read()

sample

'TST1-MUC3-0001\n\n   GUATEMALA CITY, 4 FEB 90 (ACAN-EFE) -- [TEXT] THE GUATEMALA ARMY\nDENIED TODAY THAT GUERRILLAS ATTACKED THE "SANTO TOMAS" PRESIDENTIAL\nFARM, LOCATED ON THE PACIFIC SIDE, WHERE PRESIDENT CEREZO HAS BEEN\nSTAYING SINCE 2 FEBRUARY.\n\n   A REPORT PUBLISHED BY THE "CERIGUA" NEWS AGENCY -- MOUTHPIECE OF\nTHE GUATEMALAN NATIONAL REVOLUTIONARY UNITY (URNG) -- WHOSE MAIN\nOFFICES ARE IN MEXICO, SAYS THAT A GUERRILLA COLUMN ATTACKED THE FARM\n2 DAYS AGO.\n\n   HOWEVER, ARMED FORCES SPOKESMAN COLONEL LUIS ARTURO ISAACS SAID\nTHAT THE ATTACK, WHICH RESULTED IN THE DEATH OF A CIVILIAN WHO WAS\nPASSING BY AT THE TIME OF THE SKIRMISH, WAS NOT AGAINST THE FARM, AND\nTHAT PRESIDENT CEREZO IS SAFE AND SOUND.\n\n   HE ADDED THAT ON 3 FEBRUARY PRESIDENT CEREZO MET WITH THE\nDIPLOMATIC CORPS ACCREDITED IN GUATEMALA.\n\n   THE GOVERNMENT ALSO ISSUED A COMMUNIQUE DESCRIBING THE REBEL REPORT\nAS "FALSE AND INCORRECT," AND STRESSING THAT THE PRESIDENT WAS NEVER\nIN DANGER.\n\n   COL ISA

## NLP Method 1: SPACY

In [57]:
### OPTION 1 - SPACY ###

# Run entity extraction on the sample file - SPACY

nlpd = nlp(sample)

# Ony retain entities of types we are interested in
nlpd.ents = [ent for ent in nlpd.ents   if ent.label_ in ('PERSON')]

extracted_list = list(numpy.array(nlpd.ents).flatten())


# Get extracted entities into single element format, same as gold standard -VERSION 1: 

extracted = []

for row in extracted_list:
    joined = ""
    for item in row:
        spaced = str(item) + " "
        joined += spaced
    if joined not in extracted:
        extracted.append(joined)

extracted

['CEREZO ',
 'ISAACS SAID ',
 'THEY ',
 'FLORES ',
 'GOVERNMENT TROOPS ',
 'RAMON ',
 'HECTOR LARIOS ',
 'RICARDO MEJIA ',
 'EARLY ',
 'MEJIA OVERTURNED ',
 'ALFREDO ',
 'JIMENEZ ',
 'LARIOS ',
 'LEAVING ',
 'MANAGUA ',
 'NICARAGUA ',
 'RICARDO WHEELOCK ',
 'WHEELOCK ',
 'SOME PROBLEMS ',
 'THERE ',
 'FINE ',
 'PAID ',
 'QUINTERO ',
 'GLADYS VARGAS ',
 'DAILY ',
 'SEVEN BLOCKS ',
 'ROLDAN ',
 'SEVERAL ',
 'QUINTERO AND ',
 'DRIVER ',
 'GOVERNMENT ',
 'ECONOMIC ',
 'ROBERTO CARPIO ',
 'ROBERTO \n ',
 'RETURN ',
 'COUNTRY ',
 'YAIR KLEIN IN ',
 'YAIR ',
 'FIFTH ',
 'HOW ',
 'PABLO ESCOBAR ',
 'MCLISE ',
 'ESCOBAR ',
 'ONLY ',
 'STRUGGLE ',
 'THESE CRIMINAL TERRORISTS ',
 'MOVE ',
 'DOES ',
 'PARTY ',
 'AWAIT ',
 'SALVADOR SANCHEZ CEREN ',
 'JORGE SHAFIK HANDAL ',
 'EDUARDO SANCHO ',
 'JOAQUIN VILLALOBOS ',
 'FORMER ',
 'MURDERED ',
 'RONNIE MOFFIT ',
 "VARGAS CARRENO 'S ",
 'CENTURY ',
 'ARGENTINE ',
 'VARGAS CARRENO ALSO ',
 'CARLOS MENEM ',
 'PATRICIO AYLWIN ',
 'VARGAS CARRENO ',
 'DO

## NLP Method 2: NLTK

In [60]:
### OPTION 2 - NLTK ###

### Exract all named individuals (will include MPs and others)

def get_ents(ne_tree):
    ne_in_sent = []
    for subtree in ne_tree:
        if type(subtree) == nltk.tree.Tree: # If subtree is a noun chunk, i.e. NE != "O"
            ne_label = subtree.label()
            ne_string = " ".join([token for token, pos in subtree.leaves()])
            ne_in_sent.append((ne_string, ne_label))
    # Return list of entity tuples
    return ne_in_sent


# Prep data for format required for Regex parser
tokenized_sentences = nltk.sent_tokenize(sample)

tokenized_words = [nltk.word_tokenize(sent) for sent in tokenized_sentences]

postagged_words = [nltk.pos_tag(sent) for sent in tokenized_words]

chunked=[]

for word in postagged_words:
    chunk = nltk.ne_chunk(word, binary=False)
    chunked.append(chunk)
        
ents = [get_ents(tree) for tree in chunked]
    
ents_list = list(itertools.chain.from_iterable(ents))
ents_unique = set(ents_list)

In [61]:
extracted_list = []

for row in ents_list:
    if (row[1] == 'PERSON' and row[0] not in extracted_list):
        extracted_list.append(row[0])

extracted = extracted_list

extracted

['CEREZO HAS',
 'CEREZO IS',
 'CEREZO',
 'ROBERTO CARPIO',
 'WILL MAKE',
 'ROBERTO VALLE BALDIZAN',
 'VINICIO CEREZO TO',
 'CARRY OUT',
 'CARLOS MENEM',
 'PATRICIO AYLWIN',
 'DOMINGO CAVALLO',
 'AYLWIN',
 'ALAN GARCIA',
 'GARCIA ALSO',
 'BUSH WAS',
 'VINICIO CEREZO',
 'BUSH TO EL SALVADOR',
 'ALFREDO CRISTIANI',
 'CEREZO WAS',
 'CRISTIANI AND HIMSELF',
 'CRISTIANI IS',
 'CRISTIANI BUT',
 'CRITIANI',
 'JOHN OWEN',
 'KLEIN',
 'ALBERTO MOLINA URREA',
 'GARCIA ARE TO',
 'BUSH SET',
 'BUSH WOULD',
 'BARCO',
 'BUSH FOR',
 'BUSH',
 'EDUARDO DUHALDE',
 'GEORGE BUSH',
 'MENEM TWO VOLUMES',
 'JOSE NOE',
 'CRISTIANI',
 'DENIS GUZMAN',
 'CARRY',
 'ANDRES VALLEJO',
 'JOSE LUIS LACUNZA',
 'PANAMANIAN ABDIEL',
 'GALVAN',
 'OSCAR ARIAS',
 'CARLOS HUMBERTO ROMERO',
 'CRISTIANI WHO',
 'DANIEL ORTEGA',
 'DANIEL',
 'DANIEL ORTEGA WHO',
 'IN PANAMA',
 'WALTER',
 'MR.',
 'PEREZ',
 'JOSE NAPOLEON',
 'ROBERTO',
 'ALAN GARCIA HAS',
 'CARLOS CASTANEDA',
 'LUIS',
 'AUGUSTO PINOCHET',
 'GARCIA BY',
 'AND ADMINIST

## NLP method 3: Stanford

In [64]:
### Stanford set up

import os

# Stanford imports
from nltk.tag.stanford import StanfordNERTagger
from nltk.tokenize.stanford import StanfordTokenizer

# NLTK imports
from nltk import pos_tag
from nltk.chunk import conlltags2tree
from nltk.tree import Tree



### Define paths ###

#Set core path for Stanford NLP packages
main_path = os.path.join("C:\\Users\\rothw\\Documents\\Python Scripts\\Python NLP\\StanfordNLP\\", "stanford-corenlp-full-2016-10-31\\")
# Set paths where the Standford NLP .jar files are located
pathlist = [os.path.join(main_path,"stanford-corenlp-3.7.0"),
            os.path.join(main_path,"ner\\stanford-ner.jar"),
            os.path.join(main_path,"postagger\\stanford-postagger.jar")]
###            os.path.join(main_path,"parser\\stanford-parser.jar"),
 ###            os.path.join(main_path,"parser\\stanford-parser-3.6.0-models.jar"),
    
# Set path to Stanford models
mpath = [os.path.join(main_path,"postagger\\models"), os.path.join(main_path,"ner\\classifiers")]

# Set path to java.exe
javapath = "C:\\Program Files\\Java\\jre1.8.0_121\\bin\\java.exe"

# Add paths to the CLASSPATH environmental variable (as instructed by NLTK)
os.environ['CLASSPATH'] = os.pathsep.join(pathlist)
os.environ['STANFORD_MODELS'] = os.pathsep.join(mpath)
os.environ['JAVAHOME'] = javapath



### Define function to tag NER sentence with BIO tags
def stanfordNE2BIO(tagged_sent):
    bio_tagged_sent = []
    prev_tag = "O"
    for token, tag in tagged_sent:
        if tag == "O": #O
            bio_tagged_sent.append((token, tag))
            prev_tag = tag
            continue
        if tag != "O" and prev_tag == "O": # Begin NE
            bio_tagged_sent.append((token, "B-"+tag))
            prev_tag = tag
        elif prev_tag != "O" and prev_tag == tag: # Inside NE
            bio_tagged_sent.append((token, "I-"+tag))
            prev_tag = tag
        elif prev_tag != "O" and prev_tag != tag: # Adjacent NE
            bio_tagged_sent.append((token, "B-"+tag))
            prev_tag = tag
    # Return BIO tagged sentence
    return bio_tagged_sent

In [65]:
# Run the Stanford NLP

extracted = []

# Tokenize sentence with stanford NLP
tkn_sent = StanfordTokenizer().tokenize(sample)

# Named entity tagging
tag_sent = StanfordNERTagger('english.conll.4class.distsim.crf.ser.gz').tag(tkn_sent) 

# Apply BIO tags to the tagged sentence
bio_tagged_sent = stanfordNE2BIO(tag_sent)

# Collate BIO parts of entities together
sent_tokens, sent_ne_tags = zip(*bio_tagged_sent)
sent_pos_tags = [pos for token, pos in pos_tag(sent_tokens)]

sent_conlltags = [(token, pos, ne) for token, pos, ne in zip(sent_tokens, sent_pos_tags, sent_ne_tags)]
ne_tree = conlltags2tree(sent_conlltags)

# Get entities from the trees
for subtree in ne_tree:
    if type(subtree) == Tree:
        ne_label = subtree.label()
        ne_string = " ".join([token for token, pos in subtree.leaves()])
        if ne_label in ['PERSON']:
            # extracted.append([ne_string, ne_label])
            extracted.append(ne_string)

extracted

['LUIS ARTURO ISAACS',
 'ISAACS',
 'ISAACS',
 'HECTOR',
 'RICARDO MEJIA',
 'FRANCISCO JOSE GUERRERO',
 'LARIOS',
 'LARIOS',
 'MEJIA',
 'LOPEZ',
 'ALFREDO JIMENEZ',
 'LARIOS',
 'LARIOS',
 'LARIOS',
 'LARIOS',
 'CASTILLE',
 'RICARDO WHEELOCK',
 'WALDEMAR FRANKLIN QUINTERO',
 'ANTONIO ROLDAN BETANCUR',
 'QUINTERO',
 'GLADYS VARGAS',
 'WALDEMAR FRANKLIN QUINTERO',
 'ANTONIO ROLDAN',
 'QUINTERO',
 'QUINTERO',
 'QUINTERO',
 'QUINTERO',
 'ROBERTO CARPIO',
 'ROBERTO VALLE BALDIZAN',
 'VINICIO',
 'KLEIN',
 'YAIR',
 'PABLO ESCOBAR',
 'PETER MCLISE',
 'PABLO ESCOBAR',
 'DAVID',
 'ESCOBAR',
 'ESCOBAR',
 'SALVADOR',
 'FRANCISCO JOVEL',
 'JORGE SHAFIK HANDAL',
 'EDUARDO SANCHO',
 'JOAQUIN',
 'JUAN',
 'BRYANT',
 'RONNIE MOFFIT',
 'CARRENO',
 'VARGAS CARRENO',
 'CARLOS MENEM',
 'PATRICIO AYLWIN',
 'VARGAS CARRENO',
 'DOMINGO CAVALLO',
 'RAUL CARIGNANO',
 'VARGAS CARRENO',
 'VARGAS CARRENO',
 'AYLWIN',
 'ALAN GARCIA',
 'GARCIA',
 'GARCIA',
 'ARNULFO ARIAS',
 'ARIAS',
 'NICOLAS ARDITO BARLETTA',
 'ARDIT

## Evaluation

### Format extracted entities

In [66]:
# Convert to dataframe and dedupe
extracted_df = pd.DataFrame(extracted)

extracted_df = extracted_df[0].drop_duplicates().sort_values().to_frame()


# Prep columns
extracted_df.columns = ['Entity']
extracted_df['Entity'] = extracted_df['Entity'].str.strip()
extracted_df['Predicted'] = 'Y'


# Merge and compare to gold standard
merged_df = pd.merge(goldstandard_name_df, extracted_df, on='Entity', how='outer')
merged_df = merged_df.fillna('N')


# Do actual vs. predicted crosstab matrix
pd.crosstab(merged_df['Predicted'], merged_df['Actual'])

Actual,N,Y
Predicted,,
N,0,1714
Y,428,519


In [67]:
# Calculate Coverage, Confidence and Balanced F1 measures

# true_pos = merged_df.loc[(merged_df['Actual'] == 'Y' & merged_df['Predicted'] == 'Y')]
#coverage = merged_df['Counter'].loc[merged_df['Actual'] == 'Y'].count()


true_positive = merged_df['Entity'].loc[(merged_df['Predicted']=='Y') & (merged_df['Actual']=='Y')].count()
false_negative = merged_df['Entity'].loc[(merged_df['Predicted']=='N') & (merged_df['Actual']=='Y')].count()
false_positive = merged_df['Entity'].loc[(merged_df['Predicted']=='Y') & (merged_df['Actual']=='N')].count()

coverage = true_positive / (true_positive + false_negative)
confidence = true_positive / (true_positive + false_positive)

balanced_f1 = 2 * coverage * confidence / (coverage + confidence)

print(" Coverage =  {:.1%}".format(coverage), "\n Confidence = {:.1%}".format(confidence), "\n Balanced F1 = {:.1%}".format(balanced_f1))

 Coverage =  23.2% 
 Confidence = 54.8% 
 Balanced F1 = 32.6%


# Results

SPACY:
- Coverage =  15.6% 
- Confidence = 27.9% 
- Balanced F1 = 20.0%


                    Actual	
                    N	Y
    Predicted	N	0	1889
                Y	905	350

NLTK:
- Coverage =  2.9% 
- Confidence = 44.2% 
- Balanced F1 = 5.5%

                    Actual	
                    N	Y
                
                
    Predicted   N	0	2151
                Y	86	82

Stanford:
- Coverage =  18.9% 
- Confidence = 30.8% 
- Balanced F1 = 23.4%


                    Actual	
                    N	Y
Predicted		

                N	0	1714
                
                Y	428	519
                
                
-----------------------------
Jose's Spacy code:
Precision = 15.3% (2233 ents)
Recall = 27.5% (1245 ents)
Balanced = 19.7%

## Output to CSV to investigate examples of differently classified entities

In [35]:
# Spacy version
# merged_df.to_csv("C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\DSTL preprocessed data\\classified_spacy.csv")

# NLTK version
# merged_df.to_csv("C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\DSTL preprocessed data\\classified_nltk.csv")

# Stanford version
# merged_df.to_csv("C:\\Users\\rothw\\Documents\\MUC Data\\MUC3\\DSTL preprocessed data\\classified_stanford.csv")


## Investigate some examples

In [30]:
sample_text = "SANTIAGO, 19 DEC 89 (DOMESTIC SERVICE) -- [TEXT] FOREIGN MINISTER HERNAN FELIPE ERRAZURIZ HAS REPORTED THAT THE CHILEANS INVOLVED IN THE ABDUCTION OF BRAZILIAN BUSINESSMAN ABILIO DINIZ WILL BE PROSECUTED IN BRAZIL BECAUSE THE KIDNAPPING WAS PLANNED AND PERFORMED OUTSIDE OF CHILE."

#nlp(sample_text).ents
[print(ent, ", ", ent.label_) for ent in nlp(sample_text).ents]


SANTIAGO ,  ORG
19 ,  CARDINAL
DEC ,  ORG
89 ,  CARDINAL
HERNAN FELIPE ,  PERSON


[None, None, None, None, None]

### Scrap code

In [116]:
# Check some value

merged_df.loc[(merged_df['Entity'].str.contains('ALFREDO GUTIERREZ'))]

,Entity,Actual,Predicted
72,ALFREDO GUTIERREZ,Y,N


In [28]:
bio_tagged_sent

[('TST1-MUC3-0001', 'O'),
 ('GUATEMALA', 'B-LOCATION'),
 ('CITY', 'I-LOCATION'),
 (',', 'O'),
 ('4', 'O'),
 ('FEB', 'B-ORGANIZATION'),
 ('90', 'O'),
 ('-LRB-', 'O'),
 ('ACAN-EFE', 'B-ORGANIZATION'),
 ('-RRB-', 'O'),
 ('--', 'O'),
 ('-LSB-', 'O'),
 ('TEXT', 'O'),
 ('-RSB-', 'O'),
 ('THE', 'O'),
 ('GUATEMALA', 'B-LOCATION'),
 ('ARMYDENIED', 'O'),
 ('TODAY', 'O'),
 ('THAT', 'O'),
 ('GUERRILLAS', 'O'),
 ('ATTACKED', 'O'),
 ('THE', 'O'),
 ('``', 'O'),
 ('SANTO', 'B-ORGANIZATION'),
 ('TOMAS', 'I-ORGANIZATION'),
 ("''", 'O'),
 ('PRESIDENTIALFARM', 'O'),
 (',', 'O'),
 ('LOCATED', 'O'),
 ('ON', 'O'),
 ('THE', 'O'),
 ('PACIFIC', 'B-LOCATION'),
 ('SIDE', 'O'),
 (',', 'O'),
 ('WHERE', 'O'),
 ('PRESIDENT', 'O'),
 ('CEREZO', 'B-ORGANIZATION'),
 ('HAS', 'O'),
 ('BEENSTAYING', 'O'),
 ('SINCE', 'O'),
 ('2', 'O'),
 ('FEBRUARY', 'O'),
 ('.', 'O'),
 ('A', 'O'),
 ('REPORT', 'O'),
 ('PUBLISHED', 'O'),
 ('BY', 'O'),
 ('THE', 'O'),
 ('``', 'O'),
 ('CERIGUA', 'B-ORGANIZATION'),
 ("''", 'O'),
 ('NEWS', 'O'),
 (

In [36]:
nlpd = nlp(sample)

# Ony retain entities of types we are interested in
nlpd.ents = [ent for ent in nlpd.ents   if ent.label_ in ('PERSON')]

In [37]:
nlpd.ents

(CEREZO,
 CEREZO,
 CEREZO,
 ISAACS SAID,
 ISAACS NOTED,
 GOVERNMENT TROOPS,
 RAMON,
 HECTOR LARIOS,
 RICARDO MEJIA,
 GUERRERO ANNOUNCEDLARIOS',
 MEJIA OVERTURNED,
 LOPEZ,
 ALFREDOJIMENEZ,
 KIDNAPPINGRING,
 MANAGUA,
 NICARAGUA,
 RICARDO WHEELOCK,
 WHEELOCK,
 SOME PROBLEMS,
 WHEELOCK,
 FINE,
 PAID,
 RETURNED TOTHEM.TST1-MUC3-0006,
 QUINTERO,
 ROLDAN BETANCUR,
 QUINTERO,
 QUINTERO,
 GLADYS VARGAS,
 SEVEN BLOCKS,
 QUINTERO,
 QUINTERO,
 ECONOMIC,
 ROBERTO CARPIO,
 GOVERNMENT,
 ROBERTOVALLE BALDIZAN,
 RETURN,
 YAIR KLEIN IN,
 YAIR,
 ESCOBAR,
 MCLISE,
 ONLY,
 STRUGGLE,
 THESE CRIMINAL TERRORISTS,
 RECENT HOURSHAVE,
 DOES,
 HOLD THEMEETING,
 MEETING,
 THERE,
 AWAIT,
 SALVADOR SANCHEZ CEREN,
 JORGE SHAFIK HANDAL,
 EDUARDO SANCHO,
 JOAQUIN VILLALOBOS,
 MURDERED,
 RONNIE MOFFIT,
 VARGAS CARRENO'S,
 CENTURY,
 ARGENTINE,
 CARLOS MENEM,
 AYLWIN,
 VARGAS CARRENO,
 DOMINGO CAVALLO,
 VARGAS CARRENO,
 AYLWIN,
 ARGENTINE,
 DAWN,
 GARCIA,
 GARCIA,
 HAVECALLED,
 TO,
 GARCIA,
 HERE,
 GARCIA,
 NORIEGA,
 BUSH

In [39]:
len(nlpd.ents[3])

2

In [41]:
pd.DataFrame(list(numpy.array(nlpd.ents).flatten())).head(10)

,0,1,2,3,4,5
0,CEREZO,None,None,None,None,None
1,CEREZO,None,None,None,None,None
2,CEREZO,None,None,None,None,None
3,ISAACS,SAID,None,None,None,None
4,ISAACS,NOTED,None,None,None,None
5,GOVERNMENT,TROOPS,None,None,None,None
6,RAMON,None,None,None,None,None
7,HECTOR,LARIOS,None,None,None,None
8,RICARDO,MEJIA,None,None,None,None
9,GUERRERO,ANNOUNCEDLARIOS,',None,None,None


In [43]:
def to_text(entity):
    return entity[0]
    
pd.DataFrame(list(map(to_text, list(set(nlpd.ents)))))

TypeError: unhashable type: 'spacy.tokens.span.Span'

In [49]:
len(list(nlpd.ents)[3])

2